In [1]:
#imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

Transforms to flip crop and normalize data


Image preprocessing steps to augment data

In [2]:
training_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])



Downloading the dataset and loading it using transform

In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=training_transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


NET

3 channel input (images in R G B)  
4 Convolutional layers  
2 linear  
10 outputs  

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        # Fully connected
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
         # Conv block 1
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        # Conv block 2
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        # Conv block 3
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        # Flatten (dynamic, avoids mismatch)
        x = x.view(x.size(0), -1)
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = CNN()

Loss and optimizer

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

Training loop

In [12]:
epochs = 25
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs= model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    scheduler.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(trainloader):.4f}")

Epoch [1/25], Loss: 1.5603
Epoch [2/25], Loss: 1.2124
Epoch [3/25], Loss: 1.0527
Epoch [4/25], Loss: 0.9468
Epoch [5/25], Loss: 0.8709
Epoch [6/25], Loss: 0.8208
Epoch [7/25], Loss: 0.7684
Epoch [8/25], Loss: 0.7284
Epoch [9/25], Loss: 0.6896
Epoch [10/25], Loss: 0.6568
Epoch [11/25], Loss: 0.5817
Epoch [12/25], Loss: 0.5521
Epoch [13/25], Loss: 0.5358
Epoch [14/25], Loss: 0.5195
Epoch [15/25], Loss: 0.5053
Epoch [16/25], Loss: 0.4881
Epoch [17/25], Loss: 0.4809
Epoch [18/25], Loss: 0.4642
Epoch [19/25], Loss: 0.4548
Epoch [20/25], Loss: 0.4483
Epoch [21/25], Loss: 0.4093
Epoch [22/25], Loss: 0.3938
Epoch [23/25], Loss: 0.3924
Epoch [24/25], Loss: 0.3824
Epoch [25/25], Loss: 0.3754


Test the model

In [13]:
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 85.99%
